In [ ]:
import cv2
import mediapipe as mp
import numpy as np

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, model_complexity=1, min_detection_confidence=0.5, min_tracking_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

# Load user video
cap_user = cv2.VideoCapture('cap.mp4')  # Replace with user video path

# Reference angles (these should ideally be extracted from a reference pose or set manually)
reference_angles = {
    "left_elbow": 90.0,  # Example angle, replace with actual reference values
    "right_elbow": 90.0,
    "left_shoulder": 45.0,
    "right_shoulder": 45.0
}

def calculate_angle(a, b, c):
    """Calculate the angle between three points (a, b, c) with b as the vertex."""
    a = np.array([a.x, a.y])
    b = np.array([b.x, b.y])
    c = np.array([c.x, c.y])

    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    return angle if angle <= 180.0 else 360.0 - angle

def get_pose_angles(landmarks):
    """Calculate relevant angles for pose comparison."""
    left_elbow_angle = calculate_angle(
        landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER],
        landmarks[mp_pose.PoseLandmark.LEFT_ELBOW],
        landmarks[mp_pose.PoseLandmark.LEFT_WRIST]
    )
    right_elbow_angle = calculate_angle(
        landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER],
        landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW],
        landmarks[mp_pose.PoseLandmark.RIGHT_WRIST]
    )
    left_shoulder_angle = calculate_angle(
        landmarks[mp_pose.PoseLandmark.LEFT_HIP],
        landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER],
        landmarks[mp_pose.PoseLandmark.LEFT_ELBOW]
    )
    right_shoulder_angle = calculate_angle(
        landmarks[mp_pose.PoseLandmark.RIGHT_HIP],
        landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER],
        landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW]
    )
    return {
        "left_elbow": left_elbow_angle,
        "right_elbow": right_elbow_angle,
        "left_shoulder": left_shoulder_angle,
        "right_shoulder": right_shoulder_angle
    }

while cap_user.isOpened():
    success_user, frame_user = cap_user.read()

    if not success_user:
        break

    # Pose Detection for user video
    frame_user_rgb = cv2.cvtColor(frame_user, cv2.COLOR_BGR2RGB)
    results_user = pose.process(frame_user_rgb)
    frame_user_bgr = cv2.cvtColor(frame_user_rgb, cv2.COLOR_RGB2BGR)

    # Check if landmarks are detected in the user video
    if results_user.pose_landmarks:
        mp_drawing.draw_landmarks(frame_user_bgr, results_user.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        # Get angles for the user pose
        user_angles = get_pose_angles(results_user.pose_landmarks.landmark)

        # Calculate angle differences and provide feedback if needed
        feedback = []
        threshold = 10.0  # Angle difference threshold for feedback

        for joint, ref_angle in reference_angles.items():
            user_angle = user_angles[joint]
            angle_diff = abs(ref_angle - user_angle)

            # Check if the angle difference exceeds threshold
            if angle_diff > threshold:
                feedback.append(f"Adjust your {joint.replace('_', ' ')} by {angle_diff:.2f}°.")

            # Display angle values on user frame
            user_position = (10, 30 * list(user_angles.keys()).index(joint) + 20)
            cv2.putText(frame_user_bgr, f"{joint}: {user_angle:.2f}°", user_position,
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2, cv2.LINE_AA)

        # Display feedback on user video
        for i, text in enumerate(feedback, start=1):
            cv2.putText(frame_user_bgr, text, (10, 30 * i + 120), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2, cv2.LINE_AA)

    # Show user video with feedback
    cv2.imshow('User Pose with Feedback', frame_user_bgr)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

# Release resources
cap_user.release()
cv2.destroyAllWindows()
pose.close()

ModuleNotFoundError: No module named 'mediapipe'